# Asynchronous Programming in Python

---

## Table of Contents
1. Introduction to Async Programming
2. Coroutines and async/await
3. Event Loop Basics
4. Running Coroutines
5. Tasks and Concurrent Execution
6. Gathering Results
7. Timeouts and Cancellation
8. Async Context Managers and Iterators
9. Synchronization Primitives
10. Practical Patterns
11. Key Points
12. Practice Exercises

---

## 1. Introduction to Async Programming

**Asynchronous programming** allows concurrent execution without threads.

**Key concepts:**
- **Coroutines**: Functions that can be paused and resumed
- **Event Loop**: Scheduler that runs coroutines
- **Cooperative multitasking**: Coroutines yield control voluntarily

**When to use:**
- I/O-bound tasks (network requests, file operations)
- High concurrency (thousands of connections)
- When you need to avoid thread overhead

**When NOT to use:**
- CPU-bound tasks (use multiprocessing instead)
- Simple sequential scripts

In [1]:
import asyncio
import time

# Synchronous vs Asynchronous comparison
# Synchronous - blocks while waiting
def sync_task(name, delay):
    print(f"{name}: Starting")
    time.sleep(delay)  # Blocks!
    print(f"{name}: Done")
    return f"{name} result"

# Sequential execution
start = time.time()
sync_task("Task 1", 1)
sync_task("Task 2", 1)
print(f"Sequential: {time.time() - start:.2f}s")

Task 1: Starting
Task 1: Done
Task 2: Starting
Task 2: Done
Sequential: 2.01s


In [2]:
# Asynchronous - yields control while waiting
async def async_task(name, delay):
    print(f"{name}: Starting")
    await asyncio.sleep(delay)  # Yields control!
    print(f"{name}: Done")
    return f"{name} result"

# Concurrent execution
async def main():
    start = time.time()
    # Run tasks concurrently
    await asyncio.gather(
        async_task("Task 1", 1),
        async_task("Task 2", 1)
    )
    print(f"Concurrent: {time.time() - start:.2f}s")

await main()  # In Jupyter, we can await directly

Task 1: Starting
Task 2: Starting
Task 1: Done
Task 2: Done
Concurrent: 1.00s


---

## 2. Coroutines and async/await

In [3]:
# Defining a coroutine with async def
async def greet(name):
    return f"Hello, {name}!"

# Calling a coroutine returns a coroutine object (not the result!)
coro = greet("World")
print(f"Type: {type(coro)}")
print(f"Coroutine: {coro}")

Type: <class 'coroutine'>
Coroutine: <coroutine object greet at 0x79945914a2c0>


In [4]:
# Must await to get the result
async def demo():
    result = await greet("World")
    print(result)

await demo()

Hello, World!


In [5]:
# await can only be used inside async functions
async def fetch_data(url):
    print(f"Fetching {url}...")
    await asyncio.sleep(0.5)  # Simulate network delay
    return {"url": url, "data": "some data"}

async def process_data(data):
    print(f"Processing {data}...")
    await asyncio.sleep(0.2)  # Simulate processing
    return data["data"].upper()

async def main():
    # Chain of async operations
    data = await fetch_data("http://example.com")
    result = await process_data(data)
    print(f"Result: {result}")

await main()

Fetching http://example.com...
Processing {'url': 'http://example.com', 'data': 'some data'}...
Result: SOME DATA


In [6]:
# Coroutines with arguments and return values
async def add_async(a, b):
    await asyncio.sleep(0.1)
    return a + b

async def multiply_async(a, b):
    await asyncio.sleep(0.1)
    return a * b

async def calculator():
    sum_result = await add_async(5, 3)
    product_result = await multiply_async(4, 2)
    print(f"5 + 3 = {sum_result}")
    print(f"4 * 2 = {product_result}")

await calculator()

5 + 3 = 8
4 * 2 = 8


---

## 3. Event Loop Basics

In [7]:
# The event loop is the core of async programming
# It schedules and runs coroutines

# Get the current running loop (in Jupyter)
loop = asyncio.get_running_loop()
print(f"Running loop: {loop}")
print(f"Is running: {loop.is_running()}")

Running loop: <_UnixSelectorEventLoop running=True closed=False debug=False>
Is running: True


In [8]:
# In regular Python scripts (not Jupyter), you use:
# asyncio.run(main())  # Creates and runs event loop

# Example of what asyncio.run() does internally:
"""
def run(coro):
    loop = asyncio.new_event_loop()
    try:
        return loop.run_until_complete(coro)
    finally:
        loop.close()
"""

# Low-level loop methods
async def show_loop_methods():
    loop = asyncio.get_running_loop()

    # Schedule a callback
    def callback():
        print("Callback executed!")

    loop.call_soon(callback)

    # Schedule with delay
    loop.call_later(0.1, lambda: print("Delayed callback!"))

    await asyncio.sleep(0.2)

await show_loop_methods()

Callback executed!
Delayed callback!


---

## 4. Running Coroutines

In [9]:
# Method 1: await (inside async function)
async def method1():
    result = await greet("Method 1")
    print(result)

await method1()

Hello, Method 1!


In [10]:
# Method 2: asyncio.run() (in regular scripts)
# asyncio.run(main())  # Don't run in Jupyter

# Method 3: Create a task
async def method3():
    task = asyncio.create_task(greet("Method 3"))
    result = await task
    print(result)

await method3()

Hello, Method 3!


In [11]:
# Running sync code from async context
import concurrent.futures

def blocking_io():
    """Blocking I/O operation."""
    time.sleep(0.5)
    return "blocking result"

async def run_blocking():
    loop = asyncio.get_running_loop()

    # Run in default executor (thread pool)
    result = await loop.run_in_executor(None, blocking_io)
    print(f"Result: {result}")

    # Run in custom executor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        result = await loop.run_in_executor(executor, blocking_io)
        print(f"Custom executor result: {result}")

await run_blocking()

Result: blocking result
Custom executor result: blocking result


---

## 5. Tasks and Concurrent Execution

In [12]:
# Tasks wrap coroutines for concurrent execution
async def worker(name, delay):
    print(f"{name}: starting")
    await asyncio.sleep(delay)
    print(f"{name}: done")
    return f"{name} result"

async def sequential_execution():
    """Tasks run one after another."""
    start = time.time()

    r1 = await worker("Task 1", 0.5)
    r2 = await worker("Task 2", 0.5)
    r3 = await worker("Task 3", 0.5)

    print(f"Sequential: {time.time() - start:.2f}s")
    return [r1, r2, r3]

await sequential_execution()

Task 1: starting
Task 1: done
Task 2: starting
Task 2: done
Task 3: starting
Task 3: done
Sequential: 1.50s


['Task 1 result', 'Task 2 result', 'Task 3 result']

In [13]:
async def concurrent_execution():
    """Tasks run concurrently."""
    start = time.time()

    # Create tasks - they start immediately!
    task1 = asyncio.create_task(worker("Task 1", 0.5))
    task2 = asyncio.create_task(worker("Task 2", 0.5))
    task3 = asyncio.create_task(worker("Task 3", 0.5))

    # Wait for all tasks
    r1 = await task1
    r2 = await task2
    r3 = await task3

    print(f"Concurrent: {time.time() - start:.2f}s")
    return [r1, r2, r3]

await concurrent_execution()

Task 1: starting
Task 2: starting
Task 3: starting
Task 1: done
Task 2: done
Task 3: done
Concurrent: 0.50s


['Task 1 result', 'Task 2 result', 'Task 3 result']

In [14]:
# Task properties and methods
async def demo_task_methods():
    async def long_task():
        await asyncio.sleep(1)
        return "completed"

    task = asyncio.create_task(long_task(), name="MyTask")

    print(f"Task name: {task.get_name()}")
    print(f"Is done: {task.done()}")
    print(f"Is cancelled: {task.cancelled()}")

    await asyncio.sleep(0.1)

    # Cancel the task
    task.cancel()

    try:
        await task
    except asyncio.CancelledError:
        print("Task was cancelled")

    print(f"Is cancelled: {task.cancelled()}")

await demo_task_methods()

Task name: MyTask
Is done: False
Is cancelled: False
Task was cancelled
Is cancelled: True


In [15]:
# Getting all running tasks
async def show_all_tasks():
    async def background():
        await asyncio.sleep(1)

    # Create some tasks
    t1 = asyncio.create_task(background(), name="BG1")
    t2 = asyncio.create_task(background(), name="BG2")

    # Get all tasks
    all_tasks = asyncio.all_tasks()
    for task in all_tasks:
        print(f"Task: {task.get_name()}, Done: {task.done()}")

    # Get current task
    current = asyncio.current_task()
    print(f"Current task: {current.get_name()}")

    t1.cancel()
    t2.cancel()

await show_all_tasks()

Task: Task-1, Done: False
Task: BG1, Done: False
Task: Task-22, Done: False
Task: BG2, Done: False
Current task: Task-22


---

## 6. Gathering Results

In [16]:
# asyncio.gather() - run coroutines concurrently
async def fetch(url):
    await asyncio.sleep(0.3)
    return f"Data from {url}"

async def gather_example():
    urls = ["url1", "url2", "url3"]

    # Gather runs all coroutines concurrently
    results = await asyncio.gather(
        fetch(urls[0]),
        fetch(urls[1]),
        fetch(urls[2])
    )

    # Results are in the same order as coroutines
    for url, result in zip(urls, results):
        print(f"{url}: {result}")

await gather_example()

url1: Data from url1
url2: Data from url2
url3: Data from url3


In [17]:
# Gather with return_exceptions=True
async def may_fail(x):
    if x == 2:
        raise ValueError(f"Error on {x}")
    await asyncio.sleep(0.1)
    return x * 2

async def gather_with_exceptions():
    # Without return_exceptions - first exception stops all
    try:
        results = await asyncio.gather(
            may_fail(1),
            may_fail(2),
            may_fail(3)
        )
    except ValueError as e:
        print(f"Caught: {e}")

    # With return_exceptions - exceptions are returned as results
    results = await asyncio.gather(
        may_fail(1),
        may_fail(2),
        may_fail(3),
        return_exceptions=True
    )

    for r in results:
        if isinstance(r, Exception):
            print(f"Exception: {r}")
        else:
            print(f"Result: {r}")

await gather_with_exceptions()

Caught: Error on 2
Result: 2
Exception: Error on 2
Result: 6


In [18]:
# asyncio.wait() - more control over completion
async def task_with_id(task_id, delay):
    await asyncio.sleep(delay)
    return f"Task {task_id} done"

async def wait_example():
    tasks = [
        asyncio.create_task(task_with_id(1, 0.3)),
        asyncio.create_task(task_with_id(2, 0.1)),
        asyncio.create_task(task_with_id(3, 0.2)),
    ]

    # Wait for first to complete
    done, pending = await asyncio.wait(
        tasks,
        return_when=asyncio.FIRST_COMPLETED
    )

    print(f"First completed: {done.pop().result()}")
    print(f"Still pending: {len(pending)}")

    # Wait for remaining
    done, pending = await asyncio.wait(pending)
    for task in done:
        print(f"Completed: {task.result()}")

await wait_example()

First completed: Task 2 done
Still pending: 2
Completed: Task 1 done
Completed: Task 3 done


In [19]:
# as_completed() - iterate as tasks complete
async def as_completed_example():
    async def task(n):
        await asyncio.sleep(n / 10)
        return n

    coros = [task(i) for i in [3, 1, 2]]

    # Process results as they complete
    for coro in asyncio.as_completed(coros):
        result = await coro
        print(f"Completed: {result}")

await as_completed_example()

Completed: 1
Completed: 2
Completed: 3


---

## 7. Timeouts and Cancellation

In [20]:
# asyncio.timeout() - Python 3.11+
async def slow_operation():
    await asyncio.sleep(5)
    return "done"

async def timeout_example():
    try:
        async with asyncio.timeout(1):
            result = await slow_operation()
            print(result)
    except asyncio.TimeoutError:
        print("Operation timed out!")

await timeout_example()

Operation timed out!


In [21]:
# wait_for() - simpler timeout
async def wait_for_example():
    try:
        result = await asyncio.wait_for(slow_operation(), timeout=1)
        print(result)
    except asyncio.TimeoutError:
        print("Timed out with wait_for!")

await wait_for_example()

Timed out with wait_for!


In [22]:
# Task cancellation
async def cancellable_task():
    try:
        print("Task starting...")
        await asyncio.sleep(5)
        print("Task completed!")
    except asyncio.CancelledError:
        print("Task was cancelled!")
        # Clean up resources here
        raise  # Re-raise to propagate cancellation

async def cancel_example():
    task = asyncio.create_task(cancellable_task())

    await asyncio.sleep(0.5)

    # Cancel the task
    task.cancel()

    try:
        await task
    except asyncio.CancelledError:
        print("Handled cancellation in caller")

await cancel_example()

Task starting...
Task was cancelled!
Handled cancellation in caller


In [23]:
# Shielding from cancellation
async def important_operation():
    print("Important operation starting...")
    await asyncio.sleep(0.5)
    print("Important operation done!")
    return "important result"

async def shield_example():
    async def wrapper():
        # Shield protects from outer cancellation
        try:
            result = await asyncio.shield(important_operation())
            return result
        except asyncio.CancelledError:
            print("Wrapper cancelled, but inner operation continues")
            raise

    task = asyncio.create_task(wrapper())

    await asyncio.sleep(0.1)
    task.cancel()

    try:
        await task
    except asyncio.CancelledError:
        print("Task cancelled")

    # Wait for shielded operation to complete
    await asyncio.sleep(0.5)

await shield_example()

Important operation starting...
Wrapper cancelled, but inner operation continues
Task cancelled
Important operation done!


---

## 8. Async Context Managers and Iterators

In [24]:
# Async context manager
class AsyncResource:
    def __init__(self, name):
        self.name = name

    async def __aenter__(self):
        print(f"Acquiring {self.name}...")
        await asyncio.sleep(0.1)
        print(f"{self.name} acquired")
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print(f"Releasing {self.name}...")
        await asyncio.sleep(0.1)
        print(f"{self.name} released")
        return False

    async def do_work(self):
        print(f"Working with {self.name}")

async def context_manager_demo():
    async with AsyncResource("Database") as db:
        await db.do_work()

await context_manager_demo()

Acquiring Database...
Database acquired
Working with Database
Releasing Database...
Database released


In [25]:
# Using contextlib for async context managers
from contextlib import asynccontextmanager

@asynccontextmanager
async def async_timer():
    start = time.time()
    print("Timer started")
    try:
        yield
    finally:
        elapsed = time.time() - start
        print(f"Timer: {elapsed:.2f}s")

async def timer_demo():
    async with async_timer():
        await asyncio.sleep(0.5)
        print("Doing work...")

await timer_demo()

Timer started
Doing work...
Timer: 0.50s


In [26]:
# Async iterator
class AsyncCounter:
    def __init__(self, stop):
        self.current = 0
        self.stop = stop

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self.current >= self.stop:
            raise StopAsyncIteration
        await asyncio.sleep(0.1)  # Simulate async operation
        self.current += 1
        return self.current

async def iterator_demo():
    async for num in AsyncCounter(5):
        print(f"Got: {num}")

await iterator_demo()

Got: 1
Got: 2
Got: 3
Got: 4
Got: 5


In [27]:
# Async generator (simpler way to create async iterators)
async def async_range(start, stop):
    for i in range(start, stop):
        await asyncio.sleep(0.1)
        yield i

async def generator_demo():
    async for num in async_range(1, 5):
        print(f"Generated: {num}")

await generator_demo()

Generated: 1
Generated: 2
Generated: 3
Generated: 4


In [28]:
# Async comprehensions
async def get_value(x):
    await asyncio.sleep(0.05)
    return x * 2

async def comprehension_demo():
    # Async list comprehension
    values = [await get_value(x) for x in range(5)]
    print(f"List: {values}")

    # Async generator expression
    async for v in async_range(1, 4):
        print(f"From async for: {v}")

await comprehension_demo()

List: [0, 2, 4, 6, 8]
From async for: 1
From async for: 2
From async for: 3


---

## 9. Synchronization Primitives

In [29]:
# asyncio.Lock
async def lock_demo():
    lock = asyncio.Lock()
    shared_resource = {"counter": 0}

    async def increment(name):
        for _ in range(3):
            async with lock:
                # Critical section
                current = shared_resource["counter"]
                await asyncio.sleep(0.1)  # Simulate work
                shared_resource["counter"] = current + 1
                print(f"{name}: counter = {shared_resource['counter']}")

    await asyncio.gather(
        increment("Task A"),
        increment("Task B")
    )

    print(f"Final: {shared_resource['counter']}")

await lock_demo()

Task A: counter = 1
Task B: counter = 2
Task A: counter = 3
Task B: counter = 4
Task A: counter = 5
Task B: counter = 6
Final: 6


In [30]:
# asyncio.Event
async def event_demo():
    event = asyncio.Event()

    async def waiter(name):
        print(f"{name}: Waiting for event...")
        await event.wait()
        print(f"{name}: Event received!")

    async def setter():
        await asyncio.sleep(0.5)
        print("Setting event!")
        event.set()

    await asyncio.gather(
        waiter("Waiter 1"),
        waiter("Waiter 2"),
        setter()
    )

await event_demo()

Waiter 1: Waiting for event...
Waiter 2: Waiting for event...
Setting event!
Waiter 1: Event received!
Waiter 2: Event received!


In [31]:
# asyncio.Condition
async def condition_demo():
    condition = asyncio.Condition()
    data = []

    async def producer():
        for i in range(3):
            await asyncio.sleep(0.2)
            async with condition:
                data.append(i)
                print(f"Produced: {i}")
                condition.notify()  # Wake up consumer

    async def consumer():
        consumed = 0
        while consumed < 3:
            async with condition:
                while not data:
                    await condition.wait()
                item = data.pop(0)
                print(f"Consumed: {item}")
                consumed += 1

    await asyncio.gather(producer(), consumer())

await condition_demo()

Produced: 0
Consumed: 0
Produced: 1
Consumed: 1
Produced: 2
Consumed: 2


In [32]:
# asyncio.Semaphore
async def semaphore_demo():
    # Limit concurrent access to 2
    sem = asyncio.Semaphore(2)

    async def limited_task(name):
        print(f"{name}: Waiting for semaphore...")
        async with sem:
            print(f"{name}: Acquired semaphore")
            await asyncio.sleep(0.3)
            print(f"{name}: Releasing semaphore")

    await asyncio.gather(*[
        limited_task(f"Task {i}") for i in range(5)
    ])

await semaphore_demo()

Task 0: Waiting for semaphore...
Task 0: Acquired semaphore
Task 1: Waiting for semaphore...
Task 1: Acquired semaphore
Task 2: Waiting for semaphore...
Task 3: Waiting for semaphore...
Task 4: Waiting for semaphore...
Task 0: Releasing semaphore
Task 1: Releasing semaphore
Task 2: Acquired semaphore
Task 3: Acquired semaphore
Task 2: Releasing semaphore
Task 3: Releasing semaphore
Task 4: Acquired semaphore
Task 4: Releasing semaphore


In [33]:
# asyncio.Queue
async def queue_demo():
    queue = asyncio.Queue(maxsize=3)

    async def producer():
        for i in range(5):
            await queue.put(i)
            print(f"Produced: {i}")
        await queue.put(None)  # Sentinel

    async def consumer():
        while True:
            item = await queue.get()
            if item is None:
                break
            print(f"Consumed: {item}")
            queue.task_done()

    await asyncio.gather(producer(), consumer())

await queue_demo()

Produced: 0
Produced: 1
Produced: 2
Consumed: 0
Consumed: 1
Consumed: 2
Produced: 3
Produced: 4
Consumed: 3
Consumed: 4


---

## 10. Practical Patterns

In [34]:
# Pattern 1: Concurrent HTTP requests (simulated)
async def fetch_url(url, delay=0.2):
    """Simulate HTTP request."""
    await asyncio.sleep(delay)
    return f"Data from {url}"

async def fetch_all(urls):
    """Fetch all URLs concurrently."""
    tasks = [asyncio.create_task(fetch_url(url)) for url in urls]
    results = await asyncio.gather(*tasks)
    return dict(zip(urls, results))

async def http_demo():
    urls = ["url1", "url2", "url3", "url4", "url5"]

    start = time.time()
    results = await fetch_all(urls)
    print(f"Fetched {len(results)} URLs in {time.time() - start:.2f}s")

    for url, data in results.items():
        print(f"  {url}: {data}")

await http_demo()

Fetched 5 URLs in 0.20s
  url1: Data from url1
  url2: Data from url2
  url3: Data from url3
  url4: Data from url4
  url5: Data from url5


In [35]:
# Pattern 2: Rate limiting
class RateLimiter:
    def __init__(self, rate_per_second):
        self.rate = rate_per_second
        self.semaphore = asyncio.Semaphore(rate_per_second)
        self.tokens = rate_per_second

    async def acquire(self):
        async with self.semaphore:
            await asyncio.sleep(1 / self.rate)

async def rate_limit_demo():
    limiter = RateLimiter(5)  # 5 requests per second

    async def limited_request(n):
        await limiter.acquire()
        print(f"Request {n} at {time.time():.2f}")

    start = time.time()
    await asyncio.gather(*[
        limited_request(i) for i in range(10)
    ])
    print(f"Total time: {time.time() - start:.2f}s")

await rate_limit_demo()

Request 0 at 1769891368.40
Request 1 at 1769891368.40
Request 2 at 1769891368.40
Request 3 at 1769891368.40
Request 4 at 1769891368.40
Request 5 at 1769891368.60
Request 6 at 1769891368.60
Request 7 at 1769891368.60
Request 8 at 1769891368.60
Request 9 at 1769891368.60
Total time: 0.40s


In [36]:
# Pattern 3: Worker pool
async def worker(name, queue, results):
    while True:
        item = await queue.get()
        if item is None:
            queue.task_done()
            break

        # Process item
        await asyncio.sleep(0.1)  # Simulate work
        result = item * 2
        results.append((name, item, result))
        queue.task_done()

async def worker_pool_demo():
    queue = asyncio.Queue()
    results = []
    num_workers = 3

    # Create workers
    workers = [
        asyncio.create_task(worker(f"W{i}", queue, results))
        for i in range(num_workers)
    ]

    # Add work items
    for i in range(10):
        await queue.put(i)

    # Signal workers to stop
    for _ in range(num_workers):
        await queue.put(None)

    # Wait for all work to complete
    await queue.join()
    await asyncio.gather(*workers)

    for name, item, result in sorted(results, key=lambda x: x[1]):
        print(f"{name}: {item} -> {result}")

await worker_pool_demo()

W0: 0 -> 0
W1: 1 -> 2
W2: 2 -> 4
W0: 3 -> 6
W1: 4 -> 8
W2: 5 -> 10
W0: 6 -> 12
W1: 7 -> 14
W2: 8 -> 16
W0: 9 -> 18


In [37]:
# Pattern 4: Retry with exponential backoff
async def unreliable_operation():
    import random
    if random.random() < 0.7:  # 70% failure rate
        raise Exception("Operation failed")
    return "success"

async def retry_with_backoff(coro_func, max_retries=5, base_delay=0.1):
    for attempt in range(max_retries):
        try:
            return await coro_func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            delay = base_delay * (2 ** attempt)
            print(f"Attempt {attempt + 1} failed, retrying in {delay:.2f}s")
            await asyncio.sleep(delay)

async def retry_demo():
    try:
        result = await retry_with_backoff(unreliable_operation)
        print(f"Result: {result}")
    except Exception as e:
        print(f"All retries failed: {e}")

await retry_demo()

Result: success


In [38]:
# Pattern 5: Background task management
class BackgroundTaskManager:
    def __init__(self):
        self.tasks = set()

    def create_task(self, coro):
        task = asyncio.create_task(coro)
        self.tasks.add(task)
        task.add_done_callback(self.tasks.discard)
        return task

    async def shutdown(self):
        if not self.tasks:
            return

        print(f"Cancelling {len(self.tasks)} background tasks...")
        for task in self.tasks:
            task.cancel()

        await asyncio.gather(*self.tasks, return_exceptions=True)
        print("All tasks cancelled")

async def bg_task_demo():
    manager = BackgroundTaskManager()

    async def background_work(n):
        try:
            for i in range(5):
                print(f"BG{n}: step {i}")
                await asyncio.sleep(0.2)
        except asyncio.CancelledError:
            print(f"BG{n}: cancelled")
            raise

    manager.create_task(background_work(1))
    manager.create_task(background_work(2))

    await asyncio.sleep(0.5)
    await manager.shutdown()

await bg_task_demo()

BG1: step 0
BG2: step 0
BG1: step 1
BG2: step 1
BG1: step 2
BG2: step 2
Cancelling 2 background tasks...
BG1: cancelled
BG2: cancelled
All tasks cancelled


---

## 11. Key Points

1. **async/await**: Define coroutines with `async def`, call with `await`
2. **Event Loop**: Scheduler that runs coroutines (use `asyncio.run()`)
3. **Tasks**: Wrap coroutines for concurrent execution (`create_task()`)
4. **gather()**: Run multiple coroutines concurrently, returns results in order
5. **wait()**: More control over completion conditions
6. **as_completed()**: Process results as they complete
7. **Timeouts**: Use `asyncio.timeout()` or `wait_for()`
8. **Cancellation**: Tasks can be cancelled with `task.cancel()`
9. **Sync Primitives**: Lock, Event, Condition, Semaphore, Queue
10. **Best for**: I/O-bound tasks with high concurrency needs

---

## 12. Practice Exercises

In [39]:
# Exercise 1: Concurrent file downloader (simulated)
# - Create async function to simulate downloading a file
# - Download multiple files concurrently
# - Track and display progress

async def download_file(filename, size):
    pass

async def download_all(files):
    pass

# Test:
# files = [("file1.txt", 100), ("file2.txt", 200), ("file3.txt", 150)]
# await download_all(files)

In [40]:
# Exercise 2: Async countdown timer
# - Create async countdown that prints seconds remaining
# - Run multiple timers concurrently
# - Notify when each timer completes

async def countdown(name, seconds):
    pass

# Test:
# await asyncio.gather(
#     countdown("Timer A", 3),
#     countdown("Timer B", 5)
# )

In [41]:
# Exercise 3: Producer-Consumer with async Queue
# - Multiple producers adding items to queue
# - Multiple consumers processing items
# - Use asyncio.Queue

async def producer(queue, producer_id, num_items):
    pass

async def consumer(queue, consumer_id):
    pass

# Test with 2 producers, 3 consumers

In [42]:
# Exercise 4: Rate-limited API client
# - Create client that makes at most N requests per second
# - Make 20 requests with rate limit of 5/second

class RateLimitedClient:
    def __init__(self, rate_limit):
        pass

    async def request(self, endpoint):
        pass

# Test:
# client = RateLimitedClient(5)
# tasks = [client.request(f"endpoint{i}") for i in range(20)]
# await asyncio.gather(*tasks)

In [43]:
# Exercise 5: Async web crawler (simulated)
# - Start from a URL, fetch it
# - Extract "links" (simulated) and fetch them
# - Limit concurrent requests
# - Track visited URLs

class AsyncCrawler:
    def __init__(self, max_concurrent=5):
        pass

    async def fetch(self, url):
        pass

    async def crawl(self, start_url, max_depth=2):
        pass

# Test:
# crawler = AsyncCrawler(max_concurrent=3)
# await crawler.crawl("http://example.com")

---

## Solutions

In [44]:
# Solution 1:
async def download_file(filename, size):
    print(f"Starting download: {filename} ({size} KB)")
    chunks = size // 10
    for i in range(10):
        await asyncio.sleep(0.1)  # Simulate download time
    print(f"Completed: {filename}")
    return filename

async def download_all(files):
    tasks = [download_file(name, size) for name, size in files]
    results = await asyncio.gather(*tasks)
    print(f"All downloads complete: {results}")
    return results

files = [("file1.txt", 100), ("file2.txt", 200), ("file3.txt", 150)]
await download_all(files)

Starting download: file1.txt (100 KB)
Starting download: file2.txt (200 KB)
Starting download: file3.txt (150 KB)
Completed: file1.txt
Completed: file2.txt
Completed: file3.txt
All downloads complete: ['file1.txt', 'file2.txt', 'file3.txt']


['file1.txt', 'file2.txt', 'file3.txt']

In [45]:
# Solution 2:
async def countdown(name, seconds):
    for i in range(seconds, 0, -1):
        print(f"{name}: {i}")
        await asyncio.sleep(1)
    print(f"{name}: DONE!")
    return name

await asyncio.gather(
    countdown("Timer A", 3),
    countdown("Timer B", 5)
)

Timer A: 3
Timer B: 5
Timer A: 2
Timer B: 4
Timer A: 1
Timer B: 3
Timer A: DONE!
Timer B: 2
Timer B: 1
Timer B: DONE!


['Timer A', 'Timer B']

In [46]:
# Solution 3:
async def producer(queue, producer_id, num_items):
    for i in range(num_items):
        item = f"P{producer_id}-{i}"
        await queue.put(item)
        print(f"Producer {producer_id}: produced {item}")
        await asyncio.sleep(0.1)

async def consumer(queue, consumer_id, stop_event):
    while not stop_event.is_set() or not queue.empty():
        try:
            item = await asyncio.wait_for(queue.get(), timeout=0.5)
            print(f"Consumer {consumer_id}: consumed {item}")
            queue.task_done()
        except asyncio.TimeoutError:
            continue

async def producer_consumer_demo():
    queue = asyncio.Queue()
    stop_event = asyncio.Event()

    producers = [asyncio.create_task(producer(queue, i, 3)) for i in range(2)]
    consumers = [asyncio.create_task(consumer(queue, i, stop_event)) for i in range(3)]

    await asyncio.gather(*producers)
    await queue.join()
    stop_event.set()

    for c in consumers:
        c.cancel()

await producer_consumer_demo()

Producer 0: produced P0-0
Producer 1: produced P1-0
Consumer 0: consumed P0-0
Consumer 0: consumed P1-0
Producer 0: produced P0-1
Producer 1: produced P1-1
Consumer 0: consumed P0-1
Consumer 0: consumed P1-1
Producer 0: produced P0-2
Producer 1: produced P1-2
Consumer 2: consumed P0-2
Consumer 2: consumed P1-2


In [47]:
# Solution 4:
class RateLimitedClient:
    def __init__(self, rate_limit):
        self.rate_limit = rate_limit
        self.semaphore = asyncio.Semaphore(rate_limit)

    async def request(self, endpoint):
        async with self.semaphore:
            print(f"Request to {endpoint} at {time.time():.2f}")
            await asyncio.sleep(1 / self.rate_limit)  # Enforce rate
            return f"Response from {endpoint}"

async def rate_limited_demo():
    client = RateLimitedClient(5)
    tasks = [client.request(f"endpoint{i}") for i in range(10)]

    start = time.time()
    results = await asyncio.gather(*tasks)
    print(f"Completed {len(results)} requests in {time.time() - start:.2f}s")

await rate_limited_demo()

Request to endpoint0 at 1769891375.93
Request to endpoint1 at 1769891375.93
Request to endpoint2 at 1769891375.93
Request to endpoint3 at 1769891375.93
Request to endpoint4 at 1769891375.93
Request to endpoint5 at 1769891376.13
Request to endpoint6 at 1769891376.13
Request to endpoint7 at 1769891376.13
Request to endpoint8 at 1769891376.13
Request to endpoint9 at 1769891376.13
Completed 10 requests in 0.40s


In [48]:
# Solution 5:
import random

class AsyncCrawler:
    def __init__(self, max_concurrent=5):
        self.semaphore = asyncio.Semaphore(max_concurrent)
        self.visited = set()

    async def fetch(self, url):
        async with self.semaphore:
            await asyncio.sleep(0.1)  # Simulate network delay
            # Return simulated links
            num_links = random.randint(1, 3)
            links = [f"{url}/link{i}" for i in range(num_links)]
            return {"url": url, "links": links}

    async def crawl(self, start_url, max_depth=2):
        if max_depth == 0 or start_url in self.visited:
            return

        self.visited.add(start_url)
        print(f"Crawling: {start_url}")

        result = await self.fetch(start_url)

        # Crawl discovered links concurrently
        tasks = [
            self.crawl(link, max_depth - 1)
            for link in result["links"]
            if link not in self.visited
        ]
        if tasks:
            await asyncio.gather(*tasks)

        return self.visited

crawler = AsyncCrawler(max_concurrent=3)
visited = await crawler.crawl("http://example.com", max_depth=2)
print(f"Crawled {len(visited)} URLs")

Crawling: http://example.com
Crawling: http://example.com/link0
Crawled 2 URLs
